# Consultas para realizar en FastAPI


### Importación de módulos

In [372]:
### Importación de módulos
import pandas as pd
import re
import string
import json
import ast

### Consulta 1
- def **userdata( *`User_id` : str* )**: Debe devolver cantidad de dinero gastado por el `usuario`, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.


In [108]:
# Leer dataframe de videjuegos solo con las columnas que se necesitan
columns = ['id', 'price']
games_price = pd.read_csv(r'./datasets/steam_games.csv', usecols=columns)
games_price.head()

,price,id
0,4.99,761140.0
1,Free To Play,643980.0
2,Free to Play,670290.0
3,0.99,767400.0
4,2.99,773570.0


In [109]:
# Verificar que los datatypes estén bien y no haya nulos
games_price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   price   30758 non-null  object 
 1   id      32133 non-null  float64
dtypes: float64(1), object(1)
memory usage: 502.2+ KB


In [123]:
# Función para corregir el precio, en el que aparecen valores numéricos y en formato string
def fix_price(df):

    errors_list = []
    for i in df['price']:
        try:
            float(i)
        except:
            errors_list.append(i)

    errors = set(errors_list)
    #uniques_not_free = ['Starting at $499.00', 'Starting at $449.00']
    df['price_fixed'] = df['price'].apply(lambda x: 0 if x in errors 
                                                        else 499.0 if x=='Starting at $499.00'
                                                        else 449.0 if x=='Starting at $449.00'
                                                        else x)
    df['price_fixed'] = df['price_fixed'].astype(float)
    return df

In [113]:
# Aplicar la función al dataframe
games_price = fix_price(games_price)
# Renombrar la columna de id 
games_price.rename(columns={'id':'item_id'},inplace=True)
# Eliminar la antigua columna de precio
games_price.drop(columns='price', inplace=True)
games_price.head()

In [115]:
# Verificar nulos
games_price.isna().sum()

item_id           2
price_fixed    1377
dtype: int64

In [116]:
# Reemplazar nulos en el precio con 0
games_price.fillna(0, inplace=True)

In [118]:
# Leer el daraframe de user items solo con las columnas que se necesitan
columns = ['user_id', 'item_id', 'items_count']
user_items = pd.read_csv('datasets/aus_users_items.csv', usecols=columns)
user_items.head()

,user_id,items_count,item_id
0,76561197970982479,277,10
1,76561197970982479,277,20
2,76561197970982479,277,30
3,76561197970982479,277,40
4,76561197970982479,277,50


In [119]:
# Hacer un merge (left join) de ambos dataframes on item_id
df_merged = user_items.merge(games_price, on='item_id', how='left')
df_merged.sample(10)

,user_id,items_count,item_id,price_fixed
856758,76561198008592438,294,28050,NaN
2614458,76561198064945689,155,108800,29.99
1430116,Victorypower,217,264240,9.99
5089894,76561198101205256,144,218230,0.00
4554080,76561198058959788,56,340,0.00
2974978,091263,369,70000,9.99
264034,76561198051309911,28,38410,9.99
2892329,nolongerliving,604,22230,9.99
191383,76561198080592688,242,411190,NaN
3343785,76561198079524591,66,268850,0.00


In [120]:
# Group by usuario para obtener el total de dinero gastado en videojuegos
user_spent = df_merged.groupby('user_id')['price_fixed'].agg('sum')
user_spent = pd.DataFrame(user_spent)
user_spent.reset_index()

,user_id,price_fixed
0,--000--,402.77
1,--ace--,166.82
2,--ionex--,109.92
3,-2SV-vuLB-Kg,437.49
4,-404PageNotFound-,1514.31
...,...,...
70907,zzonci,19.98
70908,zzoptimuszz,64.98
70909,zzydrax,99.94
70910,zzyfo,828.51


In [121]:
# Leer dataframe de reviews solo con las columnas que se necesitan
columns = ['user_id', 'recommend']
user_revs = pd.read_csv('./datasets/aus_user_revs_clean.csv', usecols=columns)
user_revs.sample(5)

In [123]:
# Calcular el porcentaje de recomendación para cada usuario
# Crear un df vacío
user_gb_count_revs = pd.DataFrame()
# Agrupar por user_id y agregar por conteo de recommend para obtener el total de reviews
user_gb_count_revs['count_total'] = user_revs.groupby('user_id')['recommend'].agg('count')
# Contar solo los recommend = True por usuario
user_gb_count_revs['count_true'] = user_revs[user_revs['recommend'] == True].groupby('user_id')['recommend'].agg(count_revs='count')
user_gb_count_revs.reset_index(inplace=True)
# Calcular la proporción de True entre el total
user_gb_count_revs['reco_porcentaje'] = user_gb_count_revs['count_true'] / user_gb_count_revs['count_total'] * 100
user_gb_count_revs.fillna(0,inplace=True)
user_gb_count_revs.sample(5)

In [127]:
# Hacer un merge (left join) de los df de reviews y user_spent anteriormente creados
df_merge2 = user_gb_count_revs.merge(user_spent, on='user_id',how='left' )
df_merge2.rename(columns={'price_fixed':'total_spent'},inplace=True)
df_merge2.sample(10)

In [136]:
# Se realiza otro merge con la tabla de user_items para obtener el total de items por usuario
df_merge3 = df_merge2.merge(user_items[['user_id','items_count']].drop_duplicates(),on='user_id',how='left')
# Se eliminan columnas innecesarias
df_merge3 = df_merge3.drop(['count_total','count_true'],axis=1)
# Renombrar columnas
df_merge3.rename(columns={'reco_porcentaje':'porcentaje_recomendacion', 'total_spent':'cantidad_gastada', 'items_count':'cantidad_items'},inplace=True)

In [137]:
# Guardar el dataframe final como csv para consumir en la API
df_merge3.to_csv(r'dataquery/user_data.csv',index=False)

In [138]:
# Función de la consulta para obtener el dinero total gastado, cantidad de items 
# y porcentaje de recomendación por usuario
def userdata(user_id : str):
    df_user = pd.read_csv('dataquery/user_data.csv')
    user_data = df_user[df_user['user_id'] == user_id]
    
    return user_data.to_json(orient='records')

### Consulta 2
- def **countreviews( *`YYYY-MM-DD` y `YYYY-MM-DD` : str* )**: Cantidad de usuarios que realizaron reviews entre las `fechas dadas`  y, el porcentaje de recomendación de los mismos en base a reviews.recommend.

In [411]:
#Leer dataframe de reviews solo con las columnas que se necesitan
df_counter = pd.read_csv("datasets/aus_user_revs_clean.csv", usecols=["user_id","posted","recommend"])
df_counter.sample(10)

,user_id,posted,recommend
55288,76561198085216476,"Posted March 5, 2014.",True
6517,mrfefe,Posted July 14.,True
57735,oggylous,"Posted July 16, 2014.",True
13871,somefaggotalreadytookjimmy,"Posted January 11, 2013.",True
11745,76561197981374310,"Posted October 13, 2012.",True
46770,acidqueen,"Posted March 16, 2015.",True
14693,Lioot,"Posted January 22, 2015.",True
23123,76561198202658078,"Posted September 4, 2015.",True
3842,tehbluscoot,"Posted December 17, 2014.",True
41219,76561198071067193,Posted May 14.,True


In [412]:
#Dividir la columna posted para poder extraer las fechas
df1 = df_counter["posted"].str.split(expand=True)
#Escoger las columnas útiles
df1 = df1.iloc[:,1:4]
# Concatenar al df las columnas de año, mes y día
df_counter = pd.concat([df_counter,df1], axis='columns').drop(columns=["posted"])
# Renombrar columnas
df_counter.rename(columns={1:"Month",2:"Day",3:"Year"}, inplace=True)
#Limpiar la puntuación de las columnas
df_counter["Day"] = df_counter["Day"].str.replace('[{}]'.format(string.punctuation), '',regex=True)
df_counter["Year"] = df_counter["Year"].str.replace('[{}]'.format(string.punctuation), '',regex=True)
# Reemplazar los nulos de año con la moda
df_counter['Year'].fillna(df_counter['Year'].mode()[0], inplace=True)

In [413]:
# Reemplazar los meses a valores numéricos
df_counter["Month"].replace({"January":"01","February":"02","March":"03","April":"04","May":"05",
     "June":"06","July":"07","August":"08","September":"09","October":"10","November":"11","December":"12"},inplace=True)
# Concatenar año, mes y dia en una sola columna
df_counter["Fecha"]= df_counter["Year"]+"-"+df_counter["Month"]+"-"+df_counter["Day"]
# Convertir a string
df_counter["Fecha"]=df_counter["Fecha"].astype(str)


In [407]:
# Función para convertir la columna fecha en dtype datetime
def safe_date_convert(df, date_column):
    def convert(x):
        try:
            return pd.to_datetime(x)
        except ValueError:
            # Try to find a year pattern in the string
            year_pattern = re.search(r'\b\d{4}\b', x)
            if year_pattern:
                # Convert the found year into a datetime format
                return pd.to_datetime(year_pattern.group(0), format='%Y')
            else:
                # If no year pattern is found, return NaT
                return pd.NaT

    df['date_fixed'] = df[date_column].astype(str).apply(convert)
    return df

In [414]:
df_counter = safe_date_convert(df_counter, 'Fecha')
df_counter.head()

,user_id,recommend,Month,Day,Year,Fecha,date_fixed
0,76561197970982479,True,11,5,2011,2011-11-5,2011-11-05
1,76561197970982479,True,07,15,2011,2011-07-15,2011-07-15
2,76561197970982479,True,04,21,2011,2011-04-21,2011-04-21
3,js41637,True,06,24,2014,2014-06-24,2014-06-24
4,js41637,True,09,8,2013,2013-09-8,2013-09-08


In [415]:
# Eliminar columnas sin usar y renombrar
df_counter.drop(columns=["Month","Day","Year", "Fecha"],inplace=True)
df_counter.rename(columns={'date_fixed':'Fecha'}, inplace=True)
# Guardar dataframe a archivo csv para consumir en la API
df_counter.to_csv("dataquery/count_reviews.csv",index=False)

In [416]:
# contar los reviews por fecha
print((df_counter[df_counter["Fecha"]>"2011-11-5"]["recommend"]).value_counts())
print('-'*10)
# contar los users por fecha
print(df_counter[(df_counter["Fecha"]>"2011-11-5")& (df_counter["Fecha"]<"2014-07-8")]["user_id"].nunique())
print('-'*10)
# contar las reviews recommended por fecha
recommend = df_counter[(df_counter["Fecha"]>"2011-11-5")& (df_counter["Fecha"]<"2014-07-8")]["recommend"]
print(recommend)
print('-'*10)
# calcular el porcentaje de recommended
porce_recom = (recommend.value_counts()[True])/len(recommend)
print(porce_recom)

recommend
True     52060
False     6825
Name: count, dtype: int64
----------
16079
----------
3        True
4        True
5        True
6        True
12       True
         ... 
59295    True
59296    True
59297    True
59302    True
59304    True
Name: recommend, Length: 28158, dtype: bool
----------
0.9093685631081753


In [417]:
# Funcion que calcula el porcentaje de recommended entre dos fechas determinadas
def countreviews( fecha1,fecha2 : str ):
    df_counter = pd.read_csv("dataquery/count_reviews.csv")
    cantidad_usu_rese = df_counter[(df_counter["Fecha"]>fecha1)& (df_counter["Fecha"]<fecha2)]["user_id"].nunique()
    recommend = df_counter[(df_counter["Fecha"]>fecha1)& (df_counter["Fecha"]<fecha2)]["recommend"]
    porce_recom = (recommend.value_counts()[True])/len(recommend)
    response = {'cantidad_usuarios':cantidad_usu_rese, 'porcentaje_recomendacion':porce_recom.round(2)}
    return response

In [418]:
# Probar función
countreviews('2011-11-5', '2014-07-8')

{'cantidad_usuarios': 17072, 'porcentaje_recomendacion': 0.91}

### Consulta 3
- def **genre( *`género` : str* )**:
    Devuelve el `puesto` en el que se encuentra un género sobre el ranking de los mismos analizado bajo la columna PlayTimeForever. 

In [366]:
# Leer dataframe de user_items solo con las columnas que se necesitan
columnas = ['item_id','playtime_forever']
df = pd.read_csv(r'./datasets/aus_users_items.csv',usecols=columnas)
df.head(3)

,item_id,playtime_forever
0,10,6
1,20,0
2,30,7


In [367]:
# Calcular las horas jugadas para cada videojuego
df_playhours = pd.DataFrame(df.groupby('item_id')['playtime_forever'].agg('sum')).reset_index()
df_playhours.head()

,item_id,playtime_forever
0,10,17386015
1,20,961702
2,30,758991
3,40,154486
4,50,734562


In [ ]:
# Leer dataframe de videojuegos solo con las columnas que se necesitan
columnas = ['id','genres']
df = pd.read_csv(r'./datasets/steam_games.csv',usecols=columnas)
# renombrar columnas
df.rename(columns={'id':'item_id'},inplace=True)
# Hacer merge (left join) con el df de user_items
df_merge = df_playhours.merge(df,on='item_id',how='left')
# Filtrar por los juegos que tengan playtime mayor a cero 0
df_merge = df_merge[df_merge['playtime_forever'] != 0]
# Eliminar nulos de la columna género
df_merge = df_merge.dropna(subset=['genres'])

In [ ]:
# Función para convertir a lista la columna de género
def tolist(lst):
    text= ""
    if isinstance(lst, str) :
        lst = lst.replace("0's",'0s')
        lst =  ast.literal_eval(lst)
        # try:
        #     lst =  ast.literal_eval(lst)
        # except (SyntaxError, ValueError):
        #     return lst.lower()

    return list(lst)

In [ ]:
df_merge['genres'] = df_merge['genres'].apply(lambda x : tolist(x))
df_merge.head()

In [ ]:
# Función para crear las nuevas columnas de género (tipo dummy)
def create_genre_columns(dataframe):
    genres_list = set()
    for genres in dataframe['genres']:
        if isinstance(genres, list):
            genres_list.update(genres)
    
    for genre in genres_list:
        dataframe[genre] = dataframe['genres'].apply(lambda x: 1 if genre in x else 0)
    
    return dataframe

In [ ]:
df_with_genre_columns = create_genre_columns(df_merge)
df_with_genre_columns.head()

In [ ]:
genre_columns_mapping = {
    'Racing': 'Racing',
    'Casual': 'Casual',
    'Sports': 'Sports',
    'Indie': 'Indie',
    'Web Publishing': 'Web Publishing',
    'Free to Play': 'Free to Play',
    'Software Training': 'Software Training',
    'RPG': 'RPG',
    'Early Access': 'Early Access',
    'Animation &amp; Modeling': 'Animation &amp; Modeling',
    'Strategy': 'Strategy',
    'Audio Production': 'Audio Production',
    'Action': 'Action',
    'Photo Editing': 'Photo Editing',
    'Massively Multiplayer': 'Massively Multiplayer',
    'Adventure': 'Adventure',
    'Education': 'Education',
    'Video Production': 'Video Production',
    'Simulation': 'Simulation',
    'Utilities': 'Utilities',
    'Design &amp; Illustration': 'Design &amp; Illustration'
}

# Inicializar un diccionario para almacenar las sumas de horas por género
genre_hours_sum = {}

# Iterar a través del diccionario de mapeo de columnas y calcular las sumas de horas
for col_bin, genre in genre_columns_mapping.items():
    if col_bin in df_with_genre_columns.columns:
        genre_hours_sum[genre] = df_with_genre_columns[df_with_genre_columns[col_bin] == 1]['playtime_forever'].sum()

# Crear un DataFrame con las sumas de horas por género
genre_summary_df = pd.DataFrame.from_dict(genre_hours_sum, orient='index', columns=['Total_Hours'])

# Agregar una columna para el ranking de géneros más jugados
genre_summary_df['Rank'] = genre_summary_df['Total_Hours'].rank(ascending=False).astype(int)

# Ordenar el DataFrame por las horas totales en orden descendente
genre_summary_df = genre_summary_df.sort_values(by='Total_Hours', ascending=False)

# Agregar el nombre "Genre" a la primera columna del índice
genre_summary_df = genre_summary_df.rename_axis('Genre')

# Mostrar el DataFrame con el ranking y las sumas de horas por género
genre_summary_df.to_csv(r'./dataquery/gener_rank.csv')

In [ ]:
# Leer el nuevo csv anteriormente creado
df_genre = pd.read_csv(r'./dataquery/gener_rank.csv')
# Convertir a minúsculas todos los géneros
df_genre['Genre'] = df_genre['Genre'].apply(lambda x: x.lower())
df_genre.to_csv(r'./dataquery/gener_rank.csv',index=False)

In [370]:
# Función para obtener info del género especificado
def get_genre_info(genre):
    genre = genre.lower()
    df_genre = pd.read_csv(r'./dataquery/gener_rank.csv')
    
    if df_genre['Genre'].str.contains(genre).any():
        genre_info = df_genre[df_genre['Genre']==genre]
    else:
        return 'No se encontro el genero'
    return json.dumps(genre_info.to_json(orient='records'), indent=4)

# Prueba la función con un género
genre_to_check = 'indie'
genre_info_json = get_genre_info(genre_to_check)
print(genre_info_json)

"[{\"Genre\":\"indie\",\"Total_Hours\":1494622404,\"Rank\":2}]"


### Consulta 4
def **userforgenre( *`género` : str* )**:
    `Top 5` de usuarios con más horas de juego en el género dado, con su URL (del user) y user_id.

In [ ]:
# Crear una tabla que me de las horas jugadas por genero de cada usuario
# Leer dataframe solo con columnas a importar
columnas = ['id','genres'] 
df = pd.read_csv(r'./datasets/steam_games.csv',usecols=columnas)
df.rename(columns={'id':'item_id'},inplace=True)
# remover los items que no tienen un genero definido
df = df.dropna(subset=['genres']) 

In [ ]:
# Funcion que cambia de string a formato list los objetos
def tolist(lst):
    text= ""
    if isinstance(lst, str) :
        lst = lst.replace("0's",'0s')
        lst =  ast.literal_eval(lst)

    return list(lst)
# Mapear la columna con la funcion creada
df['genres'] = df['genres'].apply(lambda x : tolist(x))

In [ ]:
# Función para crear las columnas binarias de los géneros
def create_genre_columns(dataframe):
    genres_list = set()

    # se crea el set con los valores DISTINCT de los géneros
    for genres in dataframe['genres']:
        if isinstance(genres, list):
            genres_list.update(genres)
    
    # crea una columna por cada genre en genre_list 
    # se recorre la columna genre imputando 1 cuando existe ese género 
    # en la fila analizada
    for genre in genres_list:
        dataframe[genre] = dataframe['genres'].apply(lambda x: 1 if genre in x else 0)
    
    return dataframe

In [ ]:
# Aplicar la función
df_binario = create_genre_columns(df)
# Eliminar la columna genres
df_binario.drop('genres',axis=1,inplace=True)

In [ ]:
#Importar el csv de los items por usuario
columnas = ['user_id','user_url','item_id','playtime_forever'] # Columnas a importar
df_users = pd.read_csv(r'./datasets/aus_users_items.csv',usecols=columnas)
# Merge con el df_binario recientemente modificado
df_users = df_users.merge(df_binario,on='item_id',how='left')
# Imputar nulos con cero 0
df_users.fillna(0,inplace=True)

In [ ]:
# Agrupar por usuario y calcular la suma total de tiempo jugado
col_sum = [ 'Indie','Racing','Sports','Audio Production','Animation &amp; Modeling',
            'Video Production','Software Training','Casual','Simulation','Utilities',
            'Photo Editing','Early Access','Accounting','Education',
            'Massively Multiplayer','Action','Web Publishing',
            'Adventure','Strategy','RPG','Design &amp; Illustration','Free to Play']
for col_name in col_sum:
    df_users[col_name] = df_users['playtime_forever'] * df_users[col_name]
df_users_group = df_users.groupby(['user_id','user_url'])[col_sum].agg('sum').reset_index()
df_users_group.columns = df_users_group.columns.str.lower()
display(df_users_group.head(3))


In [ ]:
# Guardar el df en un csv que consumirá la API
df_users_group.to_csv(r'./dataquery/top5_users.csv',index=False)

In [373]:
# Función que devuelve el top 5 de usuarios que más jugaron un género dado
def genre_top5_users(text : str):
    text = text.lower().strip()
    df1 = pd.read_csv(r'./dataquery/top5_users.csv')
    if text not in df1.columns:
        return "Genre not found"
    
    top5 = df1.sort_values(by=text,ascending=False).head(5).reset_index()
    response = []
    for i in range (0 , 5):
        user_id = top5['user_id'].loc[i]
        user_url = top5['user_url'].loc[i]
        item_dict = {"user_id":user_id,
                     "user_url":user_url}
        response.append(item_dict)
    return response

In [374]:
# Probar la función
genre_top5_users('rpg ')

[{'user_id': 'Evilutional',
  'user_url': 'http://steamcommunity.com/id/Evilutional'},
 {'user_id': 'shinomegami',
  'user_url': 'http://steamcommunity.com/id/shinomegami'},
 {'user_id': 'thiefofrosesinlalaland',
  'user_url': 'http://steamcommunity.com/id/thiefofrosesinlalaland'},
 {'user_id': '76561198027406146',
  'user_url': 'http://steamcommunity.com/profiles/76561198027406146'},
 {'user_id': 'KingXRay', 'user_url': 'http://steamcommunity.com/id/KingXRay'}]

### Consulta 5
def **developer( *`desarrollador` : str* )**:
    `Cantidad` de items y `porcentaje` de contenido Free por año según empresa desarrolladora. 

In [120]:
# Probar la función
sentiment_analysis('2014')

'[{"year_released":"2014","Negative":1023,"Neutral":1332.0,"Positive":4091}]'

In [254]:
# Leer dataframe con columnas necesarias
columns = ['id', 'price', 'developer','title', 'release_date']
games = pd.read_csv('datasets/steam_games.csv', usecols=columns).rename(columns={'id':'item_id'})
games.sample(10)

,title,release_date,price,item_id,developer
21524,Albino Lullaby Demo,2016-09-14,NaN,367320.0,Ape Law
3745,Flame Over,2015-05-28,11.99,345080.0,Laughing Jackal LTD
7436,Zombie Defense,2016-09-30,Free To Play,533780.0,Home Net Games
16966,NaN,NaN,4.99,601110.0,NaN
29964,Knock-knock,2013-10-04,9.99,250380.0,Ice-Pick Lodge
17449,Hacktag,2017-06-01,14.99,622770.0,Piece of Cake studios
8503,Hollow Halls,2017-01-19,2.99,576700.0,Broken Code Games
7499,SparkDimension,2016-10-04,9.99,477140.0,Asixa
12030,EVERSPACE™ - Encounters,2017-10-24,9.99,709150.0,ROCKFISH Games
26801,Rustbucket Rumble Soundtrack,2015-04-02,1.99,356780.0,Reactor Zero


In [255]:
# Transformar la columna price a numérico
games = fix_price(games)
# Transformas la columna de fechas a dtype datetime
games = safe_date_convert(games, 'release_date')
games.head()


,title,release_date,price,item_id,developer,price_fixed
0,Lost Summoner Kitty,2018-01-04,4.99,761140.0,Kotoshiro,4.99
1,Ironbound,2018-01-04,Free To Play,643980.0,Secret Level SRL,0
2,Real Pool 3D - Poolians,2017-07-24,Free to Play,670290.0,Poolians.com,0
3,弹炸人2222,2017-12-07,0.99,767400.0,彼岸领域,0.99
4,NaN,NaN,2.99,773570.0,NaN,2.99


In [ ]:
# Revisar tipos de datos y nulos
print(games.info())
print(games.isna().sum())

In [ ]:
# Obtener el año de lanzamiento
games['release_year'] = games['release_date_fixed'].dt.year
# Eliminar columnas innecesarias
games.drop(columns=['release_date','price','release_date_fixed'],inplace=True)
games.drop(columns=['title'], inplace=True)
games.head()

In [265]:
# Imputar los nulos de precio como 0 y los na de release_year por la moda
# Los nulos de desarrollador no puedo imputarlos, así que los elimino.
games.dropna(subset=['developer'], inplace=True)
games['price_fixed'].fillna(0,inplace=True)
games['release_year'].fillna(games['release_year'].mode().iloc[0],inplace=True)


In [268]:
# Verificar y eliminar duplicado
games.duplicated().sum()
games.drop_duplicates(inplace=True)

1

In [270]:
# Agrupar por desarrollador para calcular el numero de videojuegos lanzados por año
dev = games.groupby(['developer', 'release_year'], as_index=False).agg({'item_id':'count'}).rename(columns={'item_id':'item_count'})
# Calcular número de juegos gratis
free_games = games[games['price_fixed']==0].groupby(['developer','release_year'], as_index=False)['item_id'].agg('count').rename(columns={'item_id':'free'})
# Merge de los dos df (left join)
dev = pd.merge(dev, free_games, on=['developer', 'release_year'], how='left')
# Imputar nulos de la columna free con 0
dev['free'].fillna(0,inplace=True)
# Calcular porcentaje de videojuegos gratis por desarrollador por año
dev['porcentaje_free'] = dev['free'] / dev['item_count'] * 100
# Sample
dev.sample(10)

,developer,release_year,item_count,free,porcentaje_free
9637,PixelGreeds,2017.0,1,0.0,0.0
5304,Gestmorph Games,2016.0,1,1.0,100.0
9529,Phosphor Games Studio,2014.0,1,0.0,0.0
6027,ILIKESCIFI Games,2014.0,2,1.0,50.0
11747,"Spike Chunsoft Co., Ltd.",2017.0,2,0.0,0.0
7870,"Massive Damage, Inc.",2016.0,2,1.0,50.0
4483,Far Mills,2017.0,1,0.0,0.0
12283,TEN TIULENYA team,2016.0,1,0.0,0.0
11361,Sinister Design,2015.0,1,0.0,0.0
3758,Dragon Wing Studio,2017.0,6,0.0,0.0


In [273]:
# Convertir el año a entero
dev['release_year'] = dev['release_year'].astype(int)
# Convertir los nombres de desarrolladores a minúscula y eliminar espacios en blanco
dev['developer'] = dev['developer'].str.strip().str.lower()
# Redondear porcentaje de videojuegos gratis
dev['porcentaje_free'] = dev['porcentaje_free'].round(2)
# Eliminar la columna free
dev.drop(columns=['free'], inplace=True)
dev.sample(10)

,developer,release_year,item_count,porcentaje_free
11304,silver cow studio,2015,1,0.00
9136,oovee® game studios,2016,1,0.00
12922,tomatotrap,2017,1,0.00
12513,techland,2007,1,0.00
6143,illwinter game design,2012,1,0.00
11675,spark unlimited,2013,6,0.00
13813,warped tales,2017,3,33.33
10076,rex,2016,1,0.00
8454,mystik'art,2015,1,0.00
12816,thunder lotus games,2017,2,0.00


In [274]:
# Guardar el df final a un csv que consumirá la API
dev.to_csv('dataquery/developer.csv',index=False)

In [275]:
# Función que retorna la cantidad de videojuegos lanzados en un año dado
# y el porcentaje de videojuegos gratis
def developer(desarrollador : str):
    df = pd.read_csv('dataquery/developer.csv')
    desarrollador = desarrollador.strip().lower()
    if df['developer'].str.contains(desarrollador).any():
        data = df[df['developer'] == desarrollador]
        response = data[['release_year','item_count','porcentaje_free']].to_dict(orient='records')
        return response
    else:
        return 'Error: Developer not found'

In [284]:
# Probar la función
developer('Bigosaur')

[{'release_year': 2012, 'item_count': 114, 'porcentaje_free': 0.88},
 {'release_year': 2013, 'item_count': 116, 'porcentaje_free': 0.0},
 {'release_year': 2014, 'item_count': 265, 'porcentaje_free': 0.0},
 {'release_year': 2015, 'item_count': 248, 'porcentaje_free': 0.0},
 {'release_year': 2016, 'item_count': 247, 'porcentaje_free': 0.0},
 {'release_year': 2017, 'item_count': 269, 'porcentaje_free': 0.0}]

### Consulta 6
def **sentiment_analysis( *`año` : int* )**: Según el `año` de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.

- Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}

- archivo aus_user_clean, columna sentiment analysis: 0 negativo, 1 neutro, 2 positivo

In [33]:
# Leer datafame con las columnas que se necesitan
columns = ['user_id','item_id', 'sentiment_analysis']
user_revs = pd.read_csv('datasets/aus_user_revs_clean.csv', usecols=columns)
user_revs.head()

,user_id,item_id,sentiment_analysis
0,76561197970982479,1250,2.0
1,76561197970982479,22200,2.0
2,76561197970982479,43110,2.0
3,js41637,251610,2.0
4,js41637,227300,2.0


In [35]:
# Revisar tipo de datos y cantidad de nulos
print(user_revs.info())
print(user_revs.isna().sum())
# Eliminar nulos 
user_revs.dropna(inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   user_id             59305 non-null  object 
 1   item_id             59305 non-null  float64
 2   sentiment_analysis  59275 non-null  float64
dtypes: float64(2), object(1)
memory usage: 1.4+ MB


In [38]:
# Leer dataframe de videojuegos con las columnas necesarias
columns = ['id', 'release_date']
games = pd.read_csv('datasets/steam_games.csv', usecols=columns).rename(columns={'id':'item_id'})
games.sample(15)

,release_date,item_id
4283,2015-08-27,325962.0
10520,2013-06-25,669940.0
7615,2016-10-18,435000.0
25619,2015-09-09,392880.0
24878,2015-11-20,381191.0
25509,2018-02-28,406090.0
4833,2014-09-01,321060.0
9457,2005-01-01,594870.0
4042,2015-07-21,369783.0
28709,2014-07-10,279820.0


In [39]:
# Revisar tipos de datos y cantidad de nulos 
print(games.info())
print(games.isna().sum())
games.dropna(inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   release_date  30068 non-null  object 
 1   item_id       32133 non-null  float64
dtypes: float64(1), object(1)
memory usage: 502.2+ KB


In [82]:
# Hacer merge (left join) de los dos df
df_sa = user_revs.merge(games, on='item_id', how='left')
# Imputar nulos en la columna fecha con la moda
df_sa.fillna(df_sa['release_date'].mode()[0], inplace=True)
df_sa.sample(10)

,user_id,item_id,sentiment_analysis,release_date
55917,7656119815549OO43,227940.0,2.0,2016-10-18
19489,spyron,440.0,1.0,2007-10-10
11457,76561198062533699,301520.0,2.0,2017-08-24
408,76561198061174192,220200.0,2.0,2015-04-27
36825,Nz1_,236390.0,2.0,NaN
29379,certifiedweabooz,212680.0,0.0,2012-09-14
28125,Dazie,319630.0,2.0,2015-01-29
33808,76561198013785384,105600.0,1.0,2011-05-16
10013,chewyi,209650.0,1.0,2014-11-03
20285,megaglitch016,440.0,0.0,2007-10-10


In [ ]:
# Convertir la columna release_date a dtype datetime
df_sa = safe_date_convert(df_sa, 'release_date')
df_sa.info()

In [ ]:
# Obtener el año de lanzamiento
df_sa['year_released'] = df_sa['converted_date'].dt.year
# Quedarse solo con las columnas que se necesitan
df_sa = df_sa[['sentiment_analysis','year_released']]
df_sa.head()

In [102]:
# Calcular los porcentajes de reviews positivos, negativos y neutros
sentiment_analysis = pd.DataFrame()
sentiment_analysis['Negative'] = df_sa[df_sa['sentiment_analysis']==0].groupby('year_released').agg('count')
sentiment_analysis['Neutral'] = df_sa[df_sa['sentiment_analysis']==1].groupby('year_released').agg('count')
sentiment_analysis['Positive'] = df_sa[df_sa['sentiment_analysis']==2].groupby('year_released').agg('count')
sentiment_analysis.fillna(0, inplace=True)
sentiment_analysis.sample(10)

,Negative,Neutral,Positive
year_released,,,
1993,3,0.0,2
2001,8,2.0,25
2007,1590,3271.0,8031
1990,1,0.0,4
2014,1023,1332.0,4091
2005,37,75.0,182
2003,42,63.0,112
2000,8,33.0,58
1997,8,9.0,31


In [103]:
# Guardar el df final en un csv que consumirá la API
sentiment_analysis.to_csv('dataquery/sentiment_analysis.csv')

In [119]:
# Función que retorna el porcentaje de reviews negativos, positivos
# y neutros en un año especificado
def sentiment_analysis(año : str):
    df = pd.read_csv('dataquery/sentiment_analysis.csv')
    df['year_released'] = df['year_released'].astype(str)
    if df['year_released'].str.contains(año).any():
        return df[df['year_released'] == año].to_json(orient='records')
    else:
        return 'Year not found'